In [1]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names('xtreme')
print(f"XTREME 서브셋 개수: {len(xtreme_subsets)}")

XTREME 서브셋 개수: 183


In [2]:
panx_subsets = [s for s in xtreme_subsets if s.startswith('PAN')]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

- 데이터셋

In [ ]:
from datasets import load_dataset
load_dataset("xtreme", name="PAN-X.de")

In [ ]:
ko_ds = load_dataset("xtreme", name="PAN-X.ko")

In [10]:
for split in ko_ds:
    print(split)

train
validation
test


In [11]:
ko_ds['train'][:3]

{'tokens': [['현재',
   '대한민국',
   'K리그',
   '챌린지의',
   '서울',
   '이랜드',
   'FC에서',
   '활약하고',
   '있다',
   '.'],
  ["'", "''", '탄샤오', "''", "'"],
  ['개그만화', '보기', '좋은', '날+']],
 'ner_tags': [[0, 5, 3, 4, 3, 4, 4, 0, 0, 0], [0, 0, 1, 0, 0], [3, 4, 4, 4]],
 'langs': [['ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko'],
  ['ko', 'ko', 'ko', 'ko', 'ko'],
  ['ko', 'ko', 'ko', 'ko']]}

In [ ]:
from collections import defaultdict
from datasets import DatasetDict

langs = ['de', 'fr', 'it', 'en']
fracs = [0.629, 0.229, 0.084, 0.059]  # 스위스에서 사용되는 공용어 네 개의 사용 비율에 맞게 다운샘플링

# 키가 없는 경우 DatasetDict를 반환합니다.
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    
    for split in ds:
        panx_ch[lang][split] = (
            ds[split].shuffle(seed=0).select(range(int(frac*ds[split].num_rows)))
        )

In [14]:
# Dataset.num_rows 속성으로 훈련 세트에 언어마다 얼마나 많은 샘플이 담겨있는지 확인

import pandas as pd

pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs})

,de,fr,it,en
0,12580,4580,1680,1180


In [17]:
panx_ch["de"]["train"]

Dataset({
    features: ['tokens', 'ner_tags', 'langs'],
    num_rows: 12580
})

In [18]:
panx_ch["de"]["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [15]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [19]:
# ner_tags의 숫자를 태그명으로
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [21]:
panx_ch["de"]["train"].features["ner_tags"].feature

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [23]:
print(panx_ch["de"]["train"].features["ner_tags"].feature.int2str(0))
print(panx_ch["de"]["train"].features["ner_tags"].feature.int2str(1))
print(panx_ch["de"]["train"].features["ner_tags"].feature.int2str(2))

O
B-PER
I-PER


In [24]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature

In [27]:
def create_tag_names(batch):
    """
        tag 번호를 tag 이름으로 변환해 추가
    """
    return {
        "ner_tags_str": [tags.int2str(idx) for idx in batch['ner_tags']]
    }
    
panx_de = panx_ch["de"].map(create_tag_names)

Loading cached processed dataset at /home/dohye/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-c28a382859b0b828.arrow
Loading cached processed dataset at /home/dohye/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-1b544c1a25d0a20a.arrow
Loading cached processed dataset at /home/dohye/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-2766c656fc8293e8.arrow


In [28]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]], ["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [31]:
panx_de

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
        num_rows: 12580
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
        num_rows: 6290
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
        num_rows: 6290
    })
})

In [35]:
panx_de["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags_str': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [100]:
## train, validation, test 셋의 개체명 빈도를 계산해 태그가 분균형하게 부여되지 않았는지 확인
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
                
# print(split2freqs)

                
pd.DataFrame.from_dict(split2freqs, orient="index")  # orient="index" 옵션은 키가 row가 되게 해주는 옵션이다. 

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


- Counter
  - 항목을 세어준다.

In [36]:
from collections import Counter
breakfast = ['spam', 'spam', 'eggs', 'spam']  
breakfast_counter = Counter(breakfast)
breakfast_counter

Counter({'spam': 3, 'eggs': 1})

In [75]:
# Tally occurrences of words in a list
cnt = Counter()
for word in ['red', 'blue', 'red', 'green', 'blue', 'blue']:
    cnt[word] += 1
cnt


Counter({'red': 2, 'blue': 3, 'green': 1})

- 다중 언어 트랜스포머
  - 다중 언어 트랜스포머의 훈련 과정과 아키텍처는 단일 언어 프랜스포머와 비슷합니다.
  - 다만 사전 훈련에 사용하는 말뭉치가 여러 언어의 문서로 구성될 뿐입니다.
  - 언어 차이에 대한 정보가 "명시적으로" 제공되지 않아도, 이렇게 구축한 언어 표현이 여러 언어의 다양한 후속 작업에 쉽게 일반화된다는 점이 놀라운 특징입니다.
  - 교차 언어 전이를 수행하는 이런 능력이 경우에 따라 단일 언어 모델과 비슷한 결과를 내므로 언어마다 별도의 모델을 훈련할 필요가 없습니다.

- 평가 방식
  - 다중 언어 프랜스포머 모델은 일반적으로 세 가지 방법으로 평가합니다.
    - en: 영어 훈련 데이터에서 finetuning 후 각 언어의 테스트 세트에서 평가합니다.
    - each: 언어별 성능을 측정하기 위해 단일 언어의 테스트 세트에서 finetuning 후 평가합니다.
    - all: 모든 훈련 데이터에서 finetuning해 각 언어의 테스트 세트에서 평가합니다.

- XLM-RoBERTa(XLM-R)
    - 이름에 RoBERTa가 붙은 이유는 사전 훈련 방식이 단일 언어 모델 RoBERTa와 같기 때문. RoBERTa는 NSP(Next Sentence Prediction) 작업을 제거해 BERT의 성능을 크게 높였다.
    - XLM에서 사용하는 언어 임베딩을 제거하고
    - WordPiece 토크나이저 대신 100개의 언어 텍스트에서 훈련된 SentencePiece 토크나이저를 사용해 원시 텍스트를 직접 토큰화한다.
    - RoBERTa의 어휘사전 크기(토큰 수)는 55,000개지만 XML-R의 어휘사전 크기(토큰 수)는 250,000개이다. 

- XML-R 토큰화

In [78]:
# SentencePiece와 WordPiece를 비교
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [81]:
text = "Jack Sparrow loves New York!"

bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [82]:
# BERT가 문장 분류 작업에 사용하는 [CLS], [SEP] 토큰 대신에, XLM-R은 <s>와 </s>를 사용해 시퀀스의 처음과 끝을 표시한다.
pd.DataFrame([bert_tokens, xlmr_tokens], ['BERT', 'XLM-R'])

,0,1,2,3,4,5,6,7,8,9
BERT,[CLS],Jack,Spa,##rrow,loves,New,York,!,[SEP],None
XLM-R,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>


- SentencePiece 토크나이저
  - 유니그램 이라는 부분단어 분할 방식을 기반으로 각 입력 텍스트를 유니코드 문자 시퀀스로 인코딩한다.
  - 악센트, 구두점에 대해 몰라도 되기 때문에 다국어 말뭉치에 특히 유용하다. (중국어, 일본어처럼 공백이 없는 언어에도 OK)
  - 공백 문자를 "_"(U+2581)에 할당해 공백을 보존한다. 이 때문에 토큰화된 텍스트를 정확하게 원시 텍스트로 다시 변환할 수 있다.

In [83]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

'<s> Jack Sparrow loves New York!</s>'

- 트랜스포머 모델 클래스
  - huggingface transformer는 아키텍처와 작업마다 전용 클래스를 제공한다. 
    - 작업에 연관된 모델 클래스 이름은 ModelNameforTask 형식을 따른다.
    - AutoModel 클래스를 사용하는 경우 AutoModelForTask와 같다.

  - 모델 구조는 바디와 헤드로 나뉜다.
    - 헤드: 모델의 마지막 층이 후속 작업에 맞는 층으로 바뀌는데, 이 마지막 층이 모델의 헤드이며 작업에 특화된다. \<ModelName\>for\<Task\>(Ex. BertForMaskedLM, BertForSequenceClassification)
    - 바디: 헤드를 제외한 나머지 부분으로 작업에 특화되지 않은 토큰 임베딩과 트랜스포머 층이 포함된다. \<ModelName\>Model(Ex. BertModel, GPT2Model)

In [112]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig  # 새로운 모델을 초기화할 때 표준 XLM-R 설정을 사용하도록 도와준다. (RobertaPreTrainedModel의 부모 클래스인 PreTrainedModel의 클래스 변수)
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        
        # 모델 바디를 로드한다.
        self.roberta = RobertaModel(config, add_pooling_layer=False)  # add_pooling_layer의 기본값은 True로 첫 번째 토큰(여기선 [CLS])의 은닉 상태만 밀집 층과 활성화 함수에 통과시켜 출력한다. False면 [CLS] 외에 모든 은닉 상태가 반환되도록 한다.
        
        # 토큰 분류 헤드를 준비한다.
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        
        # 모델 바디에 사전 훈련된 가중치를 로드하고 토큰 분류 헤드의 가중치를 랜덤하게 초기화한다.
        self.init_weights()  # RobertaPreTrainedModel 클래스에서 상속, 이것보다는 post_init() 메서드를 사용하는 것이 좋다고 함.
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        # 모델 바디를 사용해 인코더 표현을 얻는다.
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs
        )
        
        # 인코더 표현을 헤드에 통과시킨다.
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # 손실을 계산한다.
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        # 모델 출력 객체를 반환한다.
        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
        

In [94]:
panx_de["train"]

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 12580
})

In [103]:
panx_ch["de"]["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [106]:
panx_ch["de"]["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

In [107]:
panx_ch["de"]["train"].features["ner_tags"].feature  # tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [109]:
panx_ch["de"]["train"].features["ner_tags"].feature.num_classes

7

In [108]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [110]:
from transformers import AutoConfig
xlmr_config = AutoConfig.from_pretrained(
    xlmr_model_name,
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)

loading configuration file config.json from cache at /home/dohye/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 5,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_

In [113]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, config=xlmr_config)
              .to(device))

loading weights file pytorch_model.bin from cache at /home/dohye/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification

In [114]:
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")  # encode: input_ids 만 반환
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input Ids"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input Ids,0,21763,37456,15555,5161,7,2356,5753,38,2


In [115]:
outputs = xlmr_model(input_ids.to(device)).logits

predictions = torch.argmax(outputs, dim=-1)

print(f"시퀀스에 있는 토큰 개수: {len(xlmr_tokens)}")
print(f"출력 크기: {outputs.shape}")  # batch_size, num_tokens, num_tags

시퀀스에 있는 토큰 개수: 10
출력 크기: torch.Size([1, 10, 7])


In [119]:
outputs.shape

torch.Size([1, 10, 7])

In [120]:
torch.argmax(outputs, dim=2)

tensor([[6, 4, 0, 0, 4, 4, 4, 4, 4, 6]], device='cuda:0')

In [122]:
outputs[:][0]

tensor([[ 0.1149, -0.4947, -0.1481,  0.2191,  0.2406, -0.1218,  0.5980],
        [ 0.3067, -0.6094, -0.0775,  0.0106,  0.3434, -0.1964,  0.1644],
        [ 0.2331, -0.5398, -0.0356,  0.0201,  0.2108, -0.2492,  0.2084],
        [ 0.2940, -0.5941, -0.0697, -0.0976,  0.2839, -0.2415,  0.2253],
        [ 0.1742, -0.5545, -0.0638,  0.0503,  0.3793, -0.2095,  0.1655],
        [ 0.2569, -0.6036, -0.0752,  0.0687,  0.3960, -0.2219,  0.2394],
        [ 0.2659, -0.6389, -0.1383, -0.0358,  0.4212, -0.2156,  0.2067],
        [ 0.2044, -0.6249, -0.1953, -0.0112,  0.3996, -0.2132,  0.0940],
        [ 0.2993, -0.6559, -0.1616,  0.0221,  0.3940, -0.1473,  0.2620],
        [ 0.0815, -0.5137, -0.1033,  0.2514,  0.2112, -0.1410,  0.7203]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [118]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Tags,I-LOC,I-ORG,O,O,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-LOC


In [123]:
def tag_text(text, tags, model, tokenizer):
    # 토큰 준비
    tokens = tokenizer(text).tokens()
    
    # 시퀀스를 입력 ID로 인코딩
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    
    # 가능한 7개의 클래스에 대한 로짓 출력
    outputs = model(input_ids)[0]
    
    # 토큰마다 가장 가능성이 높은 클래스 선택
    predictions = torch.argmax(outputs, dim=2)  # dim=2는 7개의 태그별 logit을 의미

    # 데이터프레임으로 변환
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]    
    
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])